In [ ]:
import math
from utils.thresholding import *

In [ ]:
def calculate_turn_angle(radius_of_curvature, wheelbase, angle_gain, is_left_turn):
    if radius_of_curvature == 0:
        radius_of_curvature = 1.0e-10
    
    turn_angle = math.atan(wheelbase/radius_of_curvature)
    
    if is_left_turn:
        turn_angle = -turn_angle
    
    return angle_gain * math.degrees(turn_angle)

In [ ]:
frame_in_w = 640
frame_in_h = 480

# Change these variables based on our car.
wheelbase = 0.1524 # Length of car from front to back in meters
n = 5 # Angle gain

# Initial radius of curvature
curvature = 1000
is_left_turn = False

ym_per_pix = 0.1524 / 72.0
xm_per_pix = 0.2286 / 600.0
y_eval = 480

In [ ]:
scale_factor_h = frame_in_w / 1280
scale_factor_v = frame_in_h / 720
offset = 200 * scale_factor_h
src = np.float32([
    [180, 280],
    [500, 280], 
    [630, 470],
    [10, 470]
])
dst = np.float32([[offset, 0], [frame_in_h - offset, 0], [frame_in_h - offset, frame_in_w], [offset, frame_in_w]])
M = cv2.getPerspectiveTransform(src,dst)

In [ ]:
def capture_frames(videoIn, frame_queue):
    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to grab frame.")
            break
        if frame_queue.full():
            # If the queue is full (only 1 frame), remove the oldest frame
            frame_queue.get()  # Discard the old frame to keep the queue size at 1
        frame_queue.put(frame)

In [ ]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

frame_queue = queue.Queue(maxsize=1)

capture_thread = threading.Thread(target=capture_frames, args=(videoIn, frame_queue))
capture_thread.daemon = True
capture_thread.start()

In [ ]:
while (True):
    if frame_queue.empty():
        continue
    
    # Get the most recent frame from the queue
    frame_vga = frame_queue.get()
    b_thresholded = threshold(frame_vga)
    binary_warped = cv2.warpPerspective(b_thresholded,M, (frame_in_w, frame_in_h))
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Finding peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Number of sliding windows
    nwindows = 9

    # Height of windows
    window_height = int(binary_warped.shape[0]/nwindows)

    # Identifing the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Margin in which to search
    margin = 100

    # Minimum number of pixels found to recenter window
    minpix = 50

    # Empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Steping through windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identifing the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # Appending these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # Recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    # Concatenating the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extracting left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    if leftx.size > 0 and lefty.size > 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)

        y1 = (2*left_fit[0]*y_eval + left_fit[1])*xm_per_pix/ym_per_pix
        y2 = 2*left_fit[0]*xm_per_pix/(ym_per_pix*ym_per_pix)

        is_left_turn = left_fit[0] < 0
        curvature = ((1 + y1*y1)**(1.5))/np.absolute(y2)
    elif rightx.size > 0 and righty.size > 0:
        # Fit a second order polynomial to each
        right_fit = np.polyfit(righty, rightx, 2)

        y1 = (2*right_fit[0]*y_eval + right_fit[1])*xm_per_pix/ym_per_pix
        y2 = 2*right_fit[0]*xm_per_pix/(ym_per_pix*ym_per_pix)

        is_left_turn = right_fit[0] < 0
        curvature = ((1 + y1*y1)**(1.5))/np.absolute(y2)
    
    angle = calculate_turn_angle(curvature, wheelbase, n, is_left_turn)
    print(angle)